# Mask Extraction

This notebook will handle converting a grayscale image mask to a WKT geometry

## Instructions

### Installation

In [8]:
from PIL import Image, ImageDraw
from IPython.display import display
from shapely.wkt import loads
import matplotlib.cm as cm

from file_utilities import nested_dict_to_json_file
from geometry_processing import convert_percentage_to_coords
from mask_extraction import get_pixel_arrays_from_image, extract_segmentations_from_pixel_array

In [53]:
IMAGE_FILE_PATH = './example/input/example_base.png'
IMAGE_MASK_FILE_PATH = './example/input/example_mask.png'
OUTPUT_PATH = f'./example/output/example_mask_wkt.json'

### Step 1: Extract pixel arrays from every image in the image folder

In [54]:
pixel_array = get_pixel_arrays_from_image(IMAGE_MASK_FILE_PATH)

### Step 2: Extract mask polygons for all voxels from pixel arrays

In [55]:
segmentation_data = extract_segmentations_from_pixel_array(
   IMAGE_MASK_FILE_PATH, pixel_array
)

### Step 3: Dump segmentation dictionary as json

In [56]:
json_file_path = nested_dict_to_json_file(segmentation_data, OUTPUT_PATH)

### Step 4: Visualize geometry on image

In [ ]:
img = Image.open(IMAGE_FILE_PATH)

width = segmentation_data['width']
height = segmentation_data['height']
cmap = cm.get_cmap('hsv')
draw = ImageDraw.Draw(img)

for seg in segmentation_data['segmentations']:
   voxel = seg['voxel']
   polygons = [loads(poly) for poly in seg['polygons']]
   for poly in polygons:
      scaled_poly = convert_percentage_to_coords(poly, width, height)
      draw.polygon(list(scaled_poly.exterior.coords), outline = cmap(voxel, bytes=True), width=2)

display(img)